In [1]:
# getting all imports out of the way first
import pandas as pd
import os
import datetime as dt

In [2]:
# Creating a function giving user options for adding transactions
def initialize():
    print("Welcome! Select what you would like to do: \n1. Enter recurring transactions\n2. Enter one-time transactions\n3. Upload a credit card statement\n4. Finish and export data\n5. Quit without saving")
    choice = int(input("Please enter a number from the menu above: "))
    
    #setting up a standard DF for recurring and one-time transactions
    recurring_df = pd.DataFrame(columns= recurring_columns)
    
    one_time_df = pd.DataFrame(columns= one_time_columns)
    # giving the user the option to manually add statements, or upload their credit card statement
    while True:
        if choice == 1:
            # Recurring transactions manual
            recurring = recurring_transactions()
            recurring_df = pd.concat([recurring, recurring_df], ignore_index=True)
            print('-'*50)
            choice = int(input("Please enter a number from the menu above "))
        
        elif choice == 2:
            # one time transactions manual
            non_recurring = non_recurring_transactions()
            one_time_df = pd.concat([one_time_df, non_recurring], ignore_index=True)
            print('-'*50)
            choice = int(input("Please enter a number from the menu above "))
        elif choice == 3:
            # Card Statement Upload
            statement = card_statement_upload()
            one_time_df = pd.concat([one_time_df, statement], ignore_index=True)
            choice = int(input("Please enter a number from the menu above "))
        elif choice == 4:
            # Exporting dataframes
            recurring_overwrite = input(print("Would you like to overwrite recurring transaction csv? (y/n) "))
            rec_output_path = "./output/recurring_transactions.csv"
            ot_output_path = "./output/one_time_transactions.csv"
            if recurring_overwrite.strip().lower() == "y":
                recurring_df.to_csv(rec_output_path, index=False)
            ot_overwrite = input(print("Would you like to overwrite one time transaction csv? (y/n) "))
        
            if ot_overwrite.strip().lower() == "y":
                one_time_df.to_csv(ot_output_path, index=False)
            print("File(s) successfully exported!")
            break
        elif choice == 5:
            # Quit without saving
            print('-'*50)
            print("Data not Saved")
            break
        else:
            print('-'*50)
            print("Please choose from the options")
            choice = int(input("Please enter a number from the menu above "))

In [3]:
recurring_columns = ["Date", "Type", "Fixed", "Sub-Categories", "Categories", "Who", "Travel", "Concept", "Amount USD", "Amount in LC", "Comments", "Exchange Rate", "Currency", "Payment Method", "Frequency", "~Monthly Cost"]


def recurring_transactions():
    recurring = pd.DataFrame(columns=recurring_columns)
    
    while True:
        start = input("Would you like to add a recurring transaction? Enter y/n: ")
        if start.strip().lower() == "n":
            break
        else:
            # Date input
            date_input = input("Enter a date (MM/DD/YYYY): ")
            try:
                date_value = pd.to_datetime(date_input, format='%m/%d/%Y')
                print("Success!")
                print(date_value)
            except:
                print("Invalid date format.")
                continue  # Skip to next iteration if date is invalid
            
            # User inputs
            sub_cat = input("Enter a sub-category: ").strip().lower().title()
            cat = input("Enter a category: ").strip().lower().title()
            user = input("Enter who pays the transaction: ").strip().lower().title()
            concept = input("Enter concept: ").strip()
            amount = float(input("Enter the transaction amount: "))
            pmt_method = input("Enter the payment method: ").strip().lower().title()
            frequency = input("Enter the frequency (e.g. Daily, weekly, monthly, yearly): ").strip().lower().title()
            
            # Calculate monthly cost based on frequency
            frequency_lower = frequency.strip().lower()
            if "weekly" in frequency_lower:
                monthly_cost = amount * 4.33  # Average weeks per month
            elif "monthly" in frequency_lower:
                monthly_cost = amount
            elif "yearly" in frequency_lower or "annual" in frequency_lower:
                monthly_cost = amount / 12
            elif "daily" in frequency_lower:
                monthly_cost = amount * 30.44  # Average days per month
            else:
                # If frequency is not in specified list, ask user or default to the amount
                monthly_cost = amount
                print(f"Unable to calculate monthly cost for frequency '{frequency}'. Using amount as-is.")
            
            # Add the transaction to the DataFrame - using the NEW column names
            new_transaction = pd.DataFrame({
                "Date": [date_value],
                "Type": ["Expenses"],
                "Fixed": [True],
                "Sub-Categories": [sub_cat],  # Changed from "Sub-Category"
                "Categories": [cat],  # Changed from "Category"
                "Who": [user],  # Changed from "User"
                "Travel": [False],
                "Concept": [concept],
                "Amount USD": [amount],  # Changed from "Amount ($)"
                "Amount in LC": [amount],
                "Comments": [""],
                "Exchange Rate": [1],
                "Currency": ["USD"],
                "Payment Method": [pmt_method],
                "Frequency": [frequency],
                "~Monthly Cost": [monthly_cost]
            })
            
            recurring = pd.concat([recurring, new_transaction], ignore_index=True)
            
            print("Recurring transaction successfully added!")
            print('-'*50)
    
    return recurring

recurring_columns = ["Type" , "Category", "Sub-Category", "User", "Amount ($)", "Frequency", "~Monthly Cost", "Payment Method"]
recurring_transactions()

In [5]:
one_time_columns = ["Date", "Type", "Fixed", "Sub-Categories", "Categories", "Who", "Travel", "Concept", "Amount USD", "Amount in LC", "Comments", "Exchange Rate", "Currency", "Payment Method", "Frequency", "~Monthly Cost"]

def non_recurring_transactions():
    non_recurring = pd.DataFrame(columns=one_time_columns)
    while True:
        start = input("Would you like to add a non-recurring transaction? Enter y/n")
        if start.strip().lower() == "n":
            break
        else: 
            date_input = input("Enter a date (MM/DD/YYYY): ")
            try:
                date_value = pd.to_datetime(date_input, format='%m/%d/%Y')
                print("Success!")
                print(date_value)
            except:
                print("Invalid date format.")
                continue  # Skip to next iteration if date is invalid
            
            sub_cat = input("Enter a sub-category: ").strip().lower().title()
            cat = input("Enter a category: ").strip().lower().title()
            user = input("Enter who made the transaction: ").strip().lower().title()
            
            # Travel input
            travel_input = input("Was this purchased while traveling? (y/n): ")
            travel = True if travel_input.strip().lower() == "y" else False
            
            concept = input("Enter concept: ").strip()
            comments = input("Enter comments (or leave blank): ").strip()
            pmt_method = input("Enter the payment method: ").strip().lower().title()
            
            # Currency input
            currency = input("Enter currency (e.g., USD, EUR, GBP): ").strip().upper()
            
            foreign = input("Is this transaction in USD ($)? Enter y/n: ")
            
            if foreign.strip().lower() == "y":
                usd = float(input("Enter the transaction amount: "))
                # For USD transactions, no foreign currency data
                amount_lc = usd
                exchange_rate = 1
            else:
                foreign_currency = float(input("Enter the transaction amount in foreign currency: "))
                exchange_rate = float(input("Enter exchange rate to USD (i.e. FC --> USD): "))
                usd = foreign_currency * exchange_rate
                amount_lc = foreign_currency
            
            # Add the transaction to the DataFrame - using the CORRECT column names
            new_transaction = pd.DataFrame({
                "Date": [date_value],
                "Type": ["Expenses"],
                "Fixed": [False],
                "Sub-Categories": [sub_cat],  # Fixed: was "Sub-Category"
                "Categories": [cat],  # Fixed: was "Category" 
                "Who": [user],  # Fixed: was "User"
                "Travel": [travel],
                "Concept": [concept],
                "Amount USD": [usd],  # Fixed: was "Amount ($)"
                "Amount in LC": [amount_lc],  # Fixed: was "Amount (LC)"
                "Comments": [comments],
                "Exchange Rate": [exchange_rate],  # Fixed: was "Exchange Rate (if applicable)"
                "Currency": [currency],
                "Payment Method": [pmt_method],
                "Frequency": [""],  # Blank
                "~Monthly Cost": [""]  # Blank
            })
            
            non_recurring = pd.concat([non_recurring, new_transaction], ignore_index=True)
            
            print("Transaction Successfully added!")
            print('-'*50)
    
    return non_recurring

one_time_columns = ["Date", "Type" , "Category", "Sub-Category", "User", "Amount ($)", "Amount (LC)", "Exchange Rate", "Payment Method"]
non_recurring_transactions()

In [20]:
def card_statement_upload():
    # defines which user this is attributed to
    user = input("Who's card statement is this?")
    print('-'*50)
    while True:
        filename = input('What is the name of the statement? (Type "back" to cancel)\nFormat must include file extension (e.g. may_c1_transactions.csv): ')
        
        if filename.lower() == "back":
            print("Operation cancelled by user.")
            return None  # Exit the function early
    
        elif filename.endswith(".csv"):
            statement = pd.read_csv(f"./statements/{filename}")
        elif filename.endswith((".xls", ".xlsx")):
            statement = pd.read_excel(f"./statements/{filename}")
        else:
            raise ValueError("File not found")
        
        print("File successfully loaded.")
        
        # Add all the new columns with proper values
        statement["Type"] = "Expenses"
        statement["Fixed"] = False
        statement["Sub-Categories"] = ""  # Blank
        statement["Who"] = user
        statement["Travel"] = ""  # Blank
        statement["Concept"] = ""  # Blank
        statement["Amount in LC"] = statement["Debit"]  # Same as Amount USD
        statement["Comments"] = ""  # Blank
        statement["Exchange Rate"] = 1
        statement["Currency"] = "USD"
        statement["Payment Method"] = "Card"
        statement["Frequency"] = ""  # Blank
        statement["~Monthly Cost"] = ""  # Blank
        
        # Clean up the dataframe
        statement = statement.dropna(subset=['Debit'])
        statement = statement.drop(['Credit', 'Posted Date', 'Card No.'], axis=1)
        
        # Rename columns to match the new structure
        statement.rename(columns={
            "Debit": "Amount USD",
            "Transaction Date": "Date",
            "Description": "Categories"  # Assuming Description becomes Categories
        }, inplace=True)
        
        # Reorder columns to match the specified order
        column_order = ["Date", "Type", "Fixed", "Sub-Categories", "Categories", "Who", "Travel", "Concept", "Amount USD", "Amount in LC", "Comments", "Exchange Rate", "Currency", "Payment Method", "Frequency", "~Monthly Cost"]
        statement = statement[column_order]
        
        return statement

In [22]:
# 03/03/2023

In [24]:
# may_c1_transactions.csv

In [26]:
recurring_columns = ["Date", "Type", "Fixed", "Sub-Categories", "Categories", "Who", "Travel", "Concept", "Amount USD", "Amount in LC", "Comments", "Exchange Rate", "Currency", "Payment Method", "Frequency", "~Monthly Cost"]
one_time_columns = ["Date", "Type", "Fixed", "Sub-Categories", "Categories", "Who", "Travel", "Concept", "Amount USD", "Amount in LC", "Comments", "Exchange Rate", "Currency", "Payment Method", "Frequency", "~Monthly Cost"]
initialize()


Welcome! Select what you would like to do: 
1. Enter recurring transactions
2. Enter one-time transactions
3. Upload a credit card statement
4. Finish and export data
5. Quit without saving


Please enter a number from the menu above:  3
Who's card statement is this? Matt


--------------------------------------------------


What is the name of the statement? (Type "back" to cancel)
Format must include file extension (e.g. may_c1_transactions.csv):  may_c1_transactions.csv


C:\Users\mattw\AppData\Local\Temp\ipykernel_5424\1372145042.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  one_time_df = pd.concat([one_time_df, statement], ignore_index=True)


...

Would you like to overwrite recurring transaction csv? (y/n) 


None y


Would you like to overwrite one time transaction csv? (y/n) 


None y


File(s) successfully exported!
